In [1]:
import pandas as pd
import numpy as np

filename = "train.txt"
with open(filename, 'r', encoding='utf-8',errors='ignore') as f:
    data = f.read()

df = pd.read_csv(pd.compat.StringIO(data),sep="\r\n", header=None)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0
0,"En route , they pick up a seemingly-harmless h..."
1,"Jobs, however, are not created by supporting a..."
2,"Due to circumstances , he is forced to abandon..."
3,I should just like to highlight two issues.\tI...
4,Jerry is finally released from hospital and re...


In [2]:
df['split'] = df[0].str.split('\t')
df['correct'] = df['split'].apply(lambda x: x[0])
df['incorrect'] = df['split'].apply(lambda x: x[1])
df.head()

,0,split,correct,incorrect
0,"En route , they pick up a seemingly-harmless h...","[En route , they pick up a seemingly-harmless ...","En route , they pick up a seemingly-harmless h...","En route , they pick up a seemingly-harmless h..."
1,"Jobs, however, are not created by supporting a...","[Jobs, however, are not created by supporting ...","Jobs, however, are not created by supporting a...","Jobs, however, are not created by supporting a..."
2,"Due to circumstances , he is forced to abandon...","[Due to circumstances , he is forced to abando...","Due to circumstances , he is forced to abandon...","Due to circumstances , he is forced to abandon..."
3,I should just like to highlight two issues.\tI...,"[I should just like to highlight two issues., ...",I should just like to highlight two issues.,I should just like to hihglight two issues.
4,Jerry is finally released from hospital and re...,[Jerry is finally released from hospital and r...,Jerry is finally released from hospital and re...,Jerry is finally released from hospital and re...


In [3]:
training_df = pd.DataFrame(df['correct'])
training_df.head()

,correct
0,"En route , they pick up a seemingly-harmless h..."
1,"Jobs, however, are not created by supporting a..."
2,"Due to circumstances , he is forced to abandon..."
3,I should just like to highlight two issues.
4,Jerry is finally released from hospital and re...


In [17]:
def convert_list_to_sentence(s):
    t = ""
    for w in s:
        t+=" "
        t+=w
    return t

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

training_list = []
for index,row in training_df.iterrows():
    text1 = nltk.word_tokenize(row['correct'])
    tag1 = [tag[1] for tag in nltk.pos_tag(text1) ]  
    training_list.append({'correct':row['correct'],'tokens':text1, 'tag':tag1})

training_df = pd.DataFrame(training_list)
training_df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,correct,tag,tokens
0,"En route , they pick up a seemingly-harmless h...","[NNP, NN, ,, PRP, VBP, RP, DT, JJ, NN, ,, CC, ...","[En, route, ,, they, pick, up, a, seemingly-ha..."
1,"Jobs, however, are not created by supporting a...","[NNP, ,, RB, ,, VBP, RB, VBN, IN, VBG, DT, JJ,...","[Jobs, ,, however, ,, are, not, created, by, s..."
2,"Due to circumstances , he is forced to abandon...","[JJ, TO, NNS, ,, PRP, VBZ, VBN, TO, VB, PRP$, ...","[Due, to, circumstances, ,, he, is, forced, to..."
3,I should just like to highlight two issues.,"[PRP, MD, RB, VB, TO, VB, CD, NNS, .]","[I, should, just, like, to, highlight, two, is..."
4,Jerry is finally released from hospital and re...,"[NNP, VBZ, RB, VBN, IN, NN, CC, NNS, TO, NNP, ...","[Jerry, is, finally, released, from, hospital,..."


In [35]:
from collections import deque

def rotate_tokens(l):
    d = deque(l)
    d.rotate(-1)
    return d

training_df['incorrect'] = training_df['tokens'].apply(lambda l: str(l[0])+convert_list_to_sentence(rotate_tokens(l[1:-1]))+str(l[-1]))
training_df.head()

,correct,tag,tokens,incorrect
0,"En route , they pick up a seemingly-harmless h...","[NNP, NN, ,, PRP, VBP, RP, DT, JJ, NN, ,, CC, ...","[En, route, ,, they, pick, up, a, seemingly-ha...","En , they pick up a seemingly-harmless hitchhi..."
1,"Jobs, however, are not created by supporting a...","[NNP, ,, RB, ,, VBP, RB, VBN, IN, VBG, DT, JJ,...","[Jobs, ,, however, ,, are, not, created, by, s...","Jobs however , are not created by supporting a..."
2,"Due to circumstances , he is forced to abandon...","[JJ, TO, NNS, ,, PRP, VBZ, VBN, TO, VB, PRP$, ...","[Due, to, circumstances, ,, he, is, forced, to...","Due circumstances , he is forced to abandon hi..."
3,I should just like to highlight two issues.,"[PRP, MD, RB, VB, TO, VB, CD, NNS, .]","[I, should, just, like, to, highlight, two, is...",I just like to highlight two issues should.
4,Jerry is finally released from hospital and re...,"[NNP, VBZ, RB, VBN, IN, NN, CC, NNS, TO, NNP, ...","[Jerry, is, finally, released, from, hospital,...",Jerry finally released from hospital and retur...


In [40]:
f1= open("result_ruin.txt","w+",encoding="utf-8")

for index,row in training_df.iterrows():
    f1.write(""+row['correct']+"\t"+row['incorrect']+"\r\n") 